# NiceCX v1.0 Tutorial

In this tutorial you will learn to use NiceCX, a simple data model that is part of the ndex2 NDEx Client module.
NiceCX facilitates creating and working with networks, including interfaces to NetworkX and Pandas.
This tutorial requires Python 3.6+ and the ndex2 module, see the NDEx Client Tutorial for installation instructions.

### Import Packages Required for this Tutorial

In [1]:
from ndex2.niceCXNetwork import NiceCXNetwork
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os

# Five Ways to Work with NiceCX networks:

- Starting with an Empty Network
- CX Files
- NDEx Networks
- NetworkX Networks
- Pandas DataFrames

## Starting with an Empty Network

**Create an empty niceCx network**

In [2]:
niceCx_creatures = NiceCXNetwork()

**Name the Network**

The _niceCx_creatures_ will now be populated with data in which each node represents a species and has a color attribute. Each edge will specify a relationship between the two species. First, we will set the name of the network:  

In [3]:
niceCx_creatures.set_name("Food Web")

**Add Nodes and Edges**

In [4]:
fox_node = niceCx_creatures.create_node(node_name='Fox')
mouse_node = niceCx_creatures.create_node(node_name='Mouse')
bird_node = niceCx_creatures.create_node(node_name='Bird')

fox_bird_edge = niceCx_creatures.create_edge(edge_source=fox_node, edge_target=bird_node, edge_interaction='interacts-with')

fox_mouse_edge = niceCx_creatures.create_edge(edge_source=fox_node, edge_target=mouse_node, edge_interaction='interacts-with')


The addNode and addEdge methods return the unique ID assigned to the new node or edge. In CX, IDs are always assigned in an ascending order, although they may not always be sequential In this case, the node with the name "Fox" will have an ID of 0, "Mouse" will have 1, and so on. The edge between "Fox" and "Bird" will have the ID of 0, the next will be 1, and so on. 

**Add Attributes**

The addNodeAttribute and addEdgeAttribute require the ID in the "property_of" field and the property "name" and "values" in their respective fields.

In [5]:
print(niceCx_creatures)
niceCx_creatures.add_node_attribute(property_of=fox_node, name='Color', values='Red')

niceCx_creatures.add_node_attribute(property_of=mouse_node, name='Color', values='Gray')

niceCx_creatures.add_node_attribute(property_of=bird_node, name='Color', values='Blue')

[{"numberVerification": [{"longNumber": 281474976710655}]}, {"nodes": [{"@id": 0, "n": "Fox"}, {"@id": 2, "n": "Mouse"}, {"@id": 1, "n": "Bird"}]}, {"edges": [{"@id": 0, "s": 0, "t": 1, "i": "interacts-with"}, {"@id": 1, "s": 0, "t": 2, "i": "interacts-with"}]}, {"networkAttributes": [{"n": "name", "d": "string", "v": "Food Web"}]}, {"metaData": [{"elementCount": 3, "version": "1.0", "consistencyGroup": 1, "name": "nodes", "idCounter": 2}, {"elementCount": 2, "version": "1.0", "consistencyGroup": 1, "name": "edges", "idCounter": 1}, {"elementCount": 1, "version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}]}]


In [6]:
niceCx_creatures.add_edge_attribute(property_of=fox_mouse_edge, name='Hunted', values='On the ground')

We can now print a summary of niceCX_creatures. The returned value of the getSummary() method is structured in the same format as NetworkSummary objects returned by NDEx network search methods.

In [7]:
print(niceCx_creatures.get_summary())

Name: Food Web
Nodes: 3
Edges: 2
Node Attributes: 3
Edge Attributes: 1



## CX Files

In [8]:
niceCx_from_cx_file = ndex2.create_nice_cx_from_file('SimpleNetwork.cx')

print(niceCx_from_cx_file.get_summary())

Name: Untitled
Nodes: 2
Edges: 1
Node Attributes: 0
Edge Attributes: 0



## NDEx Networks

In [9]:
niceCx_from_server = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')

print(niceCx_from_server.get_summary())

Name: CoCaNet2
Nodes: 36
Edges: 37
Node Attributes: 706
Edge Attributes: 517



## NetworkX Networks

Networks can be manipulated using NetworkX facilities and then used to create a NiceCx network. Here we create a NetworkX network from scratch:

In [10]:
G = nx.Graph()
G.add_node('ABC')
G.add_node('DEF')
G.add_node('GHI')
G.add_node('JKL')
G.add_node('MNO')
G.add_node('PQR')
G.add_node('XYZ')
G.add_edges_from([('ABC','DEF'), ('DEF', 'GHI'),('GHI', 'JKL'), 
                  ('DEF', 'JKL'), ('JKL', 'MNO'), ('DEF', 'MNO'),
                 ('MNO', 'XYZ'), ('DEF', 'PQR')])

We now use NetworkX to find the shortest path between nodes ABC and MNO and then create a subgraph from that path.

In [11]:
short_path = nx.shortest_path(G,source='ABC',target="MNO")

path_subgraph = G.subgraph(short_path)

Finally, NiceCx objects are created from both *G* and *path_subgraph*.

In [12]:
G.name = 'Created from NetworkX (full)'
niceCx_full = ndex2.create_nice_cx_from_networkx(G)

G.name = 'Created from NetworkX (shortest path)'
niceCx_short = ndex2.create_nice_cx_from_networkx(path_subgraph)
                       
print(niceCx_full.get_summary())
print(G.edges())
print('')

print(niceCx_short.get_summary())
print(path_subgraph.edges())

Name: Created from NetworkX (full)
Nodes: 7
Edges: 8
Node Attributes: 0
Edge Attributes: 0

[('ABC', 'DEF'), ('DEF', 'GHI'), ('DEF', 'JKL'), ('DEF', 'MNO'), ('DEF', 'PQR'), ('GHI', 'JKL'), ('JKL', 'MNO'), ('MNO', 'XYZ')]

Name: Created from NetworkX (shortest path)
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

[('ABC', 'DEF'), ('DEF', 'MNO')]


## Pandas DataFrames

### 2 Column DataFrame with No Headers:

In [13]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

niceCx_df_2_column = ndex2.create_nice_cx_from_pandas(df)

print(niceCx_df_2_column.get_summary())

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_2_col_from_niceCx = niceCx_df_2_column.to_pandas_dataframe()
print(df_2_col_from_niceCx)

Name: Pandas Upload
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

  source     interaction target
0    ABC  interacts-with    DEF
1    DEF  interacts-with    XYZ


### 3 Column DataFrame with No Headers:

In [14]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

niceCx_df_3_column = ndex2.create_nice_cx_from_pandas(df)

print(niceCx_df_3_column.get_summary())

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_3_col_from_niceCx = niceCx_df_3_column.to_pandas_dataframe()
print(df_3_col_from_niceCx)

Name: Pandas Upload
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

  source     interaction target
0    ABC  interacts-with    DEF
1    DEF     neighbor-of    XYZ


### 3+ Column DataFrame with Headers to Specify Attribute Columns

In [15]:
df = pd.DataFrame.from_items([('Source', ['ABC', 'DEF']),
                              ('Target', ['DEF', 'XYZ']),
                              ('Interaction', ['interacts-with', 'neighbor-of']),
                              ('EdgeProp', ['Edge property 1', 'Edge property 2'])])

niceCx_df_with_headers = ndex2.create_nice_cx_from_pandas(df, source_field='Source', target_field='Target', 
                          edge_attr=['EdgeProp'], edge_interaction='Interaction')

print(niceCx_df_with_headers.get_summary())

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_headers_from_niceCx = niceCx_df_with_headers.to_pandas_dataframe()
print(df_headers_from_niceCx)

Name: Pandas Upload
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 2

  source     interaction target         EdgeProp
0    ABC  interacts-with    DEF  Edge property 1
1    DEF     neighbor-of    XYZ  Edge property 2


### Exporting a Larger and More Complex Network

More complex networks can be output to a Pandas DataFrame as well. In the following example we convert the example network we loaded from the server.

In [16]:
#======================
# CONVERT TO DATAFRAME
#======================
niceCx_from_server_df = niceCx_from_server.to_pandas_dataframe()
print(niceCx_from_server_df)

       source interaction  target druggable   yeast1 known interaction  \
0   druggable          pp      TS       NaN      NaN               NaN   
1        DHFR          pp   MAPK1      DHFR  YOR236W               NaN   
2       CHEK1          pp     BLM     CHEK1  YBR274W   phosphorylation   
3      IMPDH1          pp   MTIF2    IMPDH1  YML056C               NaN   
4      IMPDH1          pp    ING5    IMPDH1  YML056C               NaN   
5      IMPDH1          pp    ING4    IMPDH1  YML056C               NaN   
6       MAPK1          pp     NF1     MAPK1  YBL016W               NaN   
7       HDAC2          pp    WEE1     HDAC2  YNL330C  synthetic lethal   
8       HDAC1          pp    WEE1     HDAC1  YNL330C  synthetic lethal   
9     CSNK1G1          pp  SH3GL1   CSNK1G1  YNL154C               NaN   
10    CSNK1G1          pp    FZR1   CSNK1G1  YNL154C               NaN   
11     MAP2K1          pp   MAPK1    MAP2K1  YJL128C               NaN   
12     MAP2K1          pp   RAD52    M

Likewise, networks can be output as a networkx graph.

In [17]:
#=====================
# CONVERT TO NETWORKX
#=====================
niceCx_from_server_to_netx = niceCx_from_server.to_networkx()

print(niceCx_from_server_to_netx)

CoCaNet2


**To continue with the tutorial, you must edit the following cell to replace the values of the ‘my_account’ and ‘my_password’ variables with a real NDEx account name and password.**

In [18]:
my_account="scratch"
my_password="scratch"
my_server="http://public.ndexbio.org"
try:
    my_ndex=nc.Ndex2(my_server, my_account, my_password)
    my_ndex.update_status()
    print("Success.  Please continue.")
except Exception as inst:
    print("Could not access account %s with password %s" % (my_account, my_password))
    print(inst.args)

Success.  Please continue.


NiceCx networks can be saved to the NDEx server by calling **upload_to()**

In [19]:
upload_message = niceCx_df_with_headers.upload_to(my_server, my_account, my_password)
print(upload_message)

http://public.ndexbio.org/v2/network/81e31d0f-6f28-11e8-a4bf-0ac135e8bacf
